# Brief analysis of the application
I implemented rice encoding based on the description from this page: https://michaeldipperstein.github.io/rice.html.
Encode and decode functions are pretty straightforward and correspond to the algorithm described on the page
above and in the course materials.
Python doesn’t have a direct method of working with separate bits with an easy and convenient approach. It only
supports byte-wise operations for integer types, which is not usable for our purposes from scratch. Among the
different options for handling separate bits, I implemented bit helpers from scratch: helper classes
RiceEncodingWriteBitContainer and RiceEncodingReadBitContainer.
I verified my implementation in 2 ways:
1. With a simple test that generates random byte arrays, encodes and decodes them, and compares the result
with the initial array.
2. I compiled the C implementation from https://github.com/michaeldipperstein/rice, ran it on the audio files, and
compared it with the encoding result of my implementation. The results are identical.
# Requested table and an analysis of the results
Below is the table with Rice encoding results.
% Compression there is calculated as round(len(encoded_bytes) / len(original_bytes) * 100).
| |Original size | Rice (K = 4 bits) | Rice (K = 2 bits) | % Compression (K = 4 bits) | % Compression (K = 2 bits) |
|-----|-----------|----------------------|-------------------|----------------------------|-----------------------------|
|Sound1.wav | 1002088 | 1516265 | 4115718 | 151 | 411|
|Sound2.wav | 1008044 | 1575347 | 4348595 | 156 | 431|

Both files have very close compression rates given the same k. That can be explained by the fact that the files have
the same type, structure and similar data profile.
But different k’s introduce a huge impact on the compression rates. k=2 provides almost 3 times worse compression
than k=4. That happens because we encode values in the range 0-255, and a lesser k requires fewer bits to encode
small numbers but many more bits to encode even a bit bigger numbers. For k=2, this border is between 23 and 24,
while for k=4, this border is already between 63 and 64. And as we have data that includes a lot of values more than
23, k=4 compresses better than k=2.
# Brief analysis of the further development implemented
As a further development, I'm trying to improve the implementation of Rice encoding algorithm.
Improvement is based on the following ideas:
1. There are a lot of 0 and 255 bytes in the audio files. So, instead of encoding them with Rice encoding, I just
use bits 10 and 11, respectively.
2. If a byte value is more than 63 for k=4, or more than 23 for k=2, its Rice-encoded representation has
significantly more bits than required without Rice encoding. In such cases, I divide the byte value by 4 and
use Rice encoding on the result. The remainder is saved separately using 2 bits. Such a format allows us to
save some space.
3. To correctly implement those ideas and distinguish them from Rice-encoded bytes, I also use some bits as
flags.
In the table below, we can see significantly improved compression rates compared to the original Rice encoding.

% Compression there is calculated as round(len(encoded_bytes) / len(original_bytes) * 100).
| |Original size | Rice (K = 4 bits) | Rice (K = 2 bits) | % Compression (K = 4 bits) | % Compression (K = 2 bits) |
|-----|-----------|----------------------|-------------------|----------------------------|-----------------------------|
|Sound1.wav | 1002088 | 891488 | 1226016 | 89 | 122|
|Sound2.wav | 1008044 | 1241172 | 1769833 | 123 | 176|

Start with installing a library for beautiful printing a resulting table.

In [43]:
!pip install tabulate

Import required dependencies.

In [44]:
import os
import random
from tabulate import tabulate

Implement a helper class for working wih bits during encoding.

In [45]:
class RiceEncodingWriteBitContainer:
    def __init__(self):
        self.byte_array = bytearray() # resulting byte array
        self.current_byte = 0 # byte used as a bits buffer
        self.current_byte_bit_position = 0 # current writing bit position in the bits buffer

    def append(self, value, number_of_bits):
        """
        Stores bits to the container.
        :param value: byte containing bits we need to store
        :param number_of_bits: how many first bits we have to store from the value param
        """
        bits_remains = number_of_bits
        while bits_remains != 0:
            bits_take = min(8 - self.current_byte_bit_position, bits_remains)
            self.current_byte |= (((value >> (bits_remains - bits_take)) & ((1 << bits_take) - 1))
                                  << (8 - self.current_byte_bit_position - bits_take))
            bits_remains -= bits_take
            self.current_byte_bit_position += bits_take
            if self.current_byte_bit_position > 7:
                self.current_byte_bit_position = 0
                self.byte_array.append(self.current_byte)
                self.current_byte = 0
                
    def to_bytearray(self):
        """
        Returns stored bits as bytearray.
        """
        result = bytearray(self.byte_array)
        if self.current_byte_bit_position != 0:
            result.append(self.current_byte | ((1 << (8 - self.current_byte_bit_position)) - 1))
        return result

    def __repr__(self):
        """
        Builds string representation of the encoded bits for debugging purposes.
        """
        return ''.join([bin(byte)[2:].zfill(8) for byte in self.to_bytearray()])

Implement a helper class for working wih bits during decoding.

In [46]:
class RiceEncodingReadBitContainer:
    def __init__(self, byte_array):
        self.byte_array = byte_array # byte array with bits we need to read
        self.bit_position = 0 # current read position in bits from byte_array start

    def read_number_of_ones(self):
        """
        Reads bits until the first 0 or until the end, and returns the number of 1 read
        """
        result = 0
        while True:
            bit = self._read_bit()
            if bit is None:
                return None
            elif bit == 1:
                result += 1
            elif bit == 0:
                return result

    def read_k_bits(self, k):
        """
        Reads k bits and returns int containing those bits
        """
        result = 0
        for i in range(k):
            result = (result << 1) | self._read_bit()
        return result
    
    def _read_bit(self):
        """
        Internal method. Reads a single bit or None if reached the end
        """
        byte_array_index = self.bit_position // 8
        if byte_array_index >= len(self.byte_array):
            return None
        result = (self.byte_array[byte_array_index] >> (7 - self.bit_position % 8)) & 1
        self.bit_position += 1
        return result

Implement Rice encoding.

In [47]:
def encode(byte_array, k):
    m = 1 << k
    result = RiceEncodingWriteBitContainer()
    
    for byte in byte_array:
        r1 = byte & (m - 1)
        r2 = (2 << (byte >> k)) - 2
        result.append(value = (r2 << k) | r1, number_of_bits = (byte >> k) + 1 + k)

    return result.to_bytearray()

Implement Rice decoding.

In [48]:
def decode(byte_array, k):
    reader = RiceEncodingReadBitContainer(byte_array)
    m = 1 << k
    result = bytearray()

    while True:
        q = reader.read_number_of_ones()
        if q is None:
            return result
        r = reader.read_k_bits(k)
        result.append(q * m + r)

Test if encoding and decoding are lossless and work correctly at least when used together.

In [49]:
for k in [2, 4]:
    print(f'Testing k={k}...')
    for i in range(1000):
        byte_array = bytearray(os.urandom(random.randint(1, 10000)))
        try:
            assert byte_array == decode(encode(byte_array, k), k)
        except:
            print(byte_array)
            raise Exception("Error!!!")

print("Everything is OK")

Testing k=2...
Testing k=4...
Everything is OK


Now it's time to apply implemented encoding/decoding to our audio files.
Start with implementing a helper function for comparison of files.

In [50]:
def have_same_content(file_1_path, file_2_path):
    with open(file_1_path, 'rb') as file1, open(file_2_path, 'rb') as file2:
        return file1.read() == file2.read()

Implement a helper function to print results as a table beautifully.

In [51]:
def print_statistics(statistics):
    print(tabulate(statistics, headers='keys'))

Run encoding -> decoding on the audio files.

In [52]:
total_statistics = []

for file in ['Sound1', 'Sound2']:
    statistics = {'File': file, 'Encoding': 'Regular'}
    
    for k in [2, 4]:
        with open(f'Exercise2_Files/{file}.wav', mode='rb') as original_sound:
            
            original_bytes = original_sound.read()
            statistics['Original size'] = len(original_bytes)
            
            encoded_bytes = encode(original_bytes, k = k)
            statistics[f'Rice (K = {k} bits)'] = len(encoded_bytes)
            statistics[f'% Compression\n(K = {k} bits)'] = round(len(encoded_bytes) / len(original_bytes) * 100)
            
            with open(f'Exercise2_Files/{file}_Enc.ex2', 'wb') as encoded_sound:
                encoded_sound.write(encoded_bytes)
    
        with open(f'Exercise2_Files/{file}_Enc.ex2', mode='rb') as encoded_sound:
            encoded_bytes = encoded_sound.read()
            with open(f'Exercise2_Files/{file}_Enc_Dec.wav', 'wb') as decoded_sound:
                decoded_sound.write(decode(encoded_bytes, k = k))

        if not have_same_content(f'Exercise2_Files/{file}.wav', f'Exercise2_Files/{file}_Enc_Dec.wav'):
            raise Exception("Encoding/decoding error!!!")

    total_statistics.append(statistics)
        
print_statistics(total_statistics)

File    Encoding      Original size    Rice (K = 2 bits)    % Compression    Rice (K = 4 bits)    % Compression
                                                             (K = 2 bits)                          (K = 4 bits)
------  ----------  ---------------  -------------------  ---------------  -------------------  ---------------
Sound1  Regular             1002088              4115718              411              1516265              151
Sound2  Regular             1008044              4348595              431              1575347              156


As a further development, I'm trying to improve the implementation of Rice encoding algorithm.
Improvement is based on the following ideas:
1. There are a lot of 0 and 255 bytes in the audio files. So, instead of encoding them with Rice encoding, I just use 10 and 11, respectively.
2. If a byte value is more than 63 for k = 4, or more than 23 for k = 2, its Rice-encoded representation has significantly more bits than  required without Rice encoding. In such case I divide the byte value by 4 and use Rice encoding on the result. The remainder is saved separately using 2 bits. Such a format allows us to save some space.
3. To correctly implement those ideas and distinguish them from Rice-encoded bytes, I also use some bits as flags.

A new helper classes for working with bits during encoding/decoding have almost the same logic but are extended with additional functionality to ease the implementation of the ideas above.

In [53]:
class EnhancedRiceEncodingWriteBitContainer(RiceEncodingWriteBitContainer):

    def to_bytearray(self):
        """
        Returns stored bits as bytearray.
        """
        result = bytearray(self.byte_array)
        if self.current_byte_bit_position != 0:
            result.append(self.current_byte | ((1 << (7 - self.current_byte_bit_position)) - 1))
        return result

In [54]:
class EnhancedRiceEncodingReadBitContainer(RiceEncodingReadBitContainer):

    def read_min_max_flag(self):
        """
        Reads and returns a flag bit that indicates if the next bit can be 0 or 255 (encoded using idea 1)
        """
        return self._read_bit()

    def read_min_max_value(self):
        """
        Reads a bit and returns 0 if the bit is 0, and 255 if the bit is 1 (according to the idea 1)
        """    
        return 255 if self._read_bit() == 1 else 0

    def read_large_value_flag(self):
        """
        Reads and returns a flag bit that indicates if the next 2 bits are actually remainder (encoded using idea 2)
        """
        return self._read_bit()

    def read_large_value_remainder(self):
        """
        Reads 2 remainder bits and returns it (according to the idea 2)
        """
        first_bit = self._read_bit()
        if first_bit is None:
            return None
        
        second_bit = self._read_bit()
        if second_bit is None:
            return None
        
        return ((0 | first_bit) << 1) | second_bit

Implement enhanced Rice encoding.

In [55]:
def enhanced_encode(byte_array, k):
    m = 1 << k
    result = EnhancedRiceEncodingWriteBitContainer()

    for byte in byte_array:
        if byte == 0:
            result.append(2, 2)
            continue
        if byte == 255:
            result.append(3, 2)
            continue
        result.append(0, 1)
        if k == 4 and byte > 63 or k == 2 and byte > 23:
            result.append(1, 1)
            result.append(byte % 4, 2)
            byte = byte // 4
        else:
            result.append(0, 1)
        r1 = byte & (m - 1)
        r2 = (2 << (byte >> k)) - 2
        result.append(value = (r2 << k) | r1, number_of_bits = (byte >> k) + 1 + k)
      
    return result.to_bytearray()

Implement enhanced Rice decoding.

In [56]:
def enhanced_decode(byte_array, k):
    reader = EnhancedRiceEncodingReadBitContainer(byte_array)
    m = 1 << k
    result = bytearray()
    
    while True:
        min_max_flag = reader.read_min_max_flag()
        if min_max_flag is None:
            return result
        if min_max_flag == 1:
            result.append(reader.read_min_max_value())
            continue
        
        large_value_flag = reader.read_large_value_flag()
        if large_value_flag is None:
            return result
        if large_value_flag == 1:
            large_value_remainder = reader.read_large_value_remainder()
        else:
            large_value_remainder = 0
        if large_value_remainder is None:
            return result
        
        q = reader.read_number_of_ones()
        if q is None:
            return result
        r = reader.read_k_bits(k)
        
        value = q * m + r
        if large_value_flag:
            value = value * 4 + large_value_remainder
        
        result.append(value)

Test if enhanced encoding and decoding are lossless and work correctly at least when used together.

In [57]:
import os
import random

for k in [2, 4]:
    print(f'Testing k={k}...')
    for i in range(1000):
        byte_array = bytearray(os.urandom(random.randint(1, 10000)))
        try:
            assert byte_array == enhanced_decode(enhanced_encode(byte_array, k), k)
        except:
            print(byte_array)
            raise Exception("Encoding/decoding error!!!")

print("Everything is OK")

Testing k=2...
Testing k=4...
Everything is OK


Run enhanced encoding -> decoding on the audio files.

In [58]:
for file in ['Sound1', 'Sound2']:
    statistics = {'File': file, 'Encoding': 'Enhanced'}
    
    for k in [2, 4]:
        with open(f'Exercise2_Files/{file}.wav', mode='rb') as original_sound:

            original_bytes = original_sound.read()
            statistics['Original size'] = len(original_bytes)

            encoded_bytes = enhanced_encode(original_bytes, k = k)
            statistics[f'Rice (K = {k} bits)'] = len(encoded_bytes)
            statistics[f'% Compression\n(K = {k} bits)'] = round(len(encoded_bytes) / len(original_bytes) * 100)

            with open(f'Exercise2_Files/{file}_Enc_Enhanced.ex2', 'wb') as encoded_sound:
                encoded_sound.write(encoded_bytes)

        with open(f'Exercise2_Files/{file}_Enc_Enhanced.ex2', mode='rb') as encoded_sound:
            encoded_bytes = encoded_sound.read()
            with open(f'Exercise2_Files/{file}_Enc_Dec_Enhanced.wav', 'wb') as decoded_sound:
                decoded_sound.write(enhanced_decode(encoded_bytes, k = k))

        if not have_same_content(f'Exercise2_Files/{file}.wav', f'Exercise2_Files/{file}_Enc_Dec_Enhanced.wav'):
            raise Exception("Encoding/decoding error!!!")

    total_statistics.append(statistics)
    
print_statistics(total_statistics)

File    Encoding      Original size    Rice (K = 2 bits)    % Compression    Rice (K = 4 bits)    % Compression
                                                             (K = 2 bits)                          (K = 4 bits)
------  ----------  ---------------  -------------------  ---------------  -------------------  ---------------
Sound1  Regular             1002088              4115718              411              1516265              151
Sound2  Regular             1008044              4348595              431              1575347              156
Sound1  Enhanced            1002088              1226016              122               891488               89
Sound2  Enhanced            1008044              1769833              176              1241172              123
